# Day 0

In [1]:
import $ivy.`org.scalaz::scalaz-core:7.2.22`
import scalaz._
import Scalaz._

import $ivy.$                               

import scalaz._

import Scalaz._

## パラメータ多相

In [2]:
def head[A](xs: List[A]): A = xs(0)

defined function head

In [3]:
head(1 :: 2 :: Nil)

res2: Int = 1

In [4]:
case class Car(make: String)

defined class Car

In [5]:
head(Car("Civic") :: Car("CR-V") :: Nil)

res4: Car = Car("Civic")

## 派生型多態

In [6]:
def plus[A](a1: A, a2: A): A = ???

defined function plus

In [7]:
trait Plus[A] {
    def plus(a2: A): A
}

defined trait Plus

In [8]:
def plus[A <: Plus[A]](a1: A, a2: A): A = a1.plus(a2)

defined function plus

## アドホック多相

In [9]:
trait Plus[A] {
    def plus(a1: A, a2: A): A
}

defined trait Plus

In [10]:
def plus[A: Plus](a1: A, a2: A): A = implicitly[Plus[A]].plus(a1, a2)

defined function plus

## sum 関数

In [11]:
def sum(xs: List[Int]): Int = xs.foldLeft(0) { _ + _ }

defined function sum

In [12]:
sum(List(1, 2, 3, 4))

res11: Int = 10

## Monoid

In [13]:
object IntMonoid {
    def mappend(a: Int, b: Int): Int = a + b
    def mzero: Int = 0
}

defined object IntMonoid

In [14]:
def sum(xs: List[Int]): Int = xs.foldLeft(IntMonoid.mzero)(IntMonoid.mappend)

defined function sum

In [15]:
sum(List(1, 2, 3, 4))

res14: Int = 10

In [16]:
trait Monoid[A] {
    def mappend(a1: A, a2: A): A
    def mzero: A
}

defined trait Monoid

In [17]:
object IntMonoid extends Monoid[Int] {
    def mappend(a: Int, b: Int): Int = a + b
    def mzero: Int = 0
}

defined object IntMonoid

In [18]:
def sum(xs: List[Int], m: Monoid[Int]): Int = xs.foldLeft(m.mzero)(m.mappend)

defined function sum

In [19]:
sum(List(1, 2, 3, 4), IntMonoid)

res18: Int = 10

In [20]:
def sum[A](xs: List[A], m: Monoid[A]): A = xs.foldLeft(m.mzero)(m.mappend)

defined function sum

In [21]:
sum(List(1, 2, 3, 4), IntMonoid)

res20: Int = 10

In [22]:
def sum[A](xs: List[A])(implicit m: Monoid[A]): A = xs.foldLeft(m.mzero)(m.mappend)

defined function sum

In [23]:
implicit val intMonoid = IntMonoid

intMonoid: IntMonoid = $sess.cmd16Wrapper$Helper$IntMonoid$@3a378fab

In [24]:
sum(List(1, 2, 3, 4))

res23: Int = 10

In [25]:
def sum[A: Monoid](xs: List[A]): A = {
    val m = implicitly[Monoid[A]]
    xs.foldLeft(m.mzero)(m.mappend)
}

defined function sum

In [26]:
sum(List(1, 2, 3, 4))

res25: Int = 10

In [27]:
trait Monoid[A] {
    def mappend(a1: A, a2: A): A
    def mzero: A
}

object Monoid {
    implicit val IntMonoid: Monoid[Int] = new Monoid[Int] {
        def mappend(a: Int, b: Int): Int = a + b
        def mzero: Int = 0
    }
    implicit val StringMonoid: Monoid[String] = new Monoid[String] {
        def mappend(a: String, b: String): String = a + b
        def mzero: String = ""
    }
}

def sum[A: Monoid](xs: List[A]): A = {
    val m = implicitly[Monoid[A]]
    xs.foldLeft(m.mzero)(m.mappend)
}

defined trait Monoid
defined object Monoid
defined function sum

In [28]:
sum(List("a", "b", "c"))

res27: String = "abc"

In [29]:
val multiMonoid: Monoid[Int] = new Monoid[Int] {
    def mappend(a: Int, b: Int): Int = a * b
    def mzero: Int = 1
}

multiMonoid: Monoid[Int] = $sess.cmd28Wrapper$Helper$$anon$1@213b5b65

In [30]:
sum(List(1, 2, 3, 4))(multiMonoid)

res29: Int = 24

## FoldLef

In [31]:
object FoldLeftList {
    def foldLeft[A, B](xs: List[A], b: B, f: (B, A) => B) = xs.foldLeft(b)(f)
}

defined object FoldLeftList

In [32]:
def sum[A: Monoid](xs: List[A]): A = {
    val m = implicitly[Monoid[A]]
    FoldLeftList.foldLeft(xs, m.mzero, m.mappend)
}

defined function sum

In [33]:
sum(List(1, 2, 3, 4))

res32: Int = 10

In [34]:
sum(List("a", "b", "c"))

res33: String = "abc"

In [35]:
sum(List(1, 2, 3, 4))(multiMonoid)

res34: Int = 24

In [36]:
trait FoldLeft[F[_]] {
    def foldLeft[A, B](xs: F[A], b: B, f: (B, A) => B): B
}

object FoldLeft {
    implicit val FoldLeftList: FoldLeft[List] = new FoldLeft[List] {
        def foldLeft[A, B](xs: List[A], b: B, f: (B, A) => B) = xs.foldLeft(b)(f)
    }
}

def sum[M[_]: FoldLeft, A: Monoid](xs: M[A]): A = {
    val m = implicitly[Monoid[A]]
    val fl = implicitly[FoldLeft[M]]
    fl.foldLeft(xs, m.mzero, m.mappend)
}

defined trait FoldLeft
defined object FoldLeft
defined function sum

In [37]:
sum(List(1, 2, 3, 4))

res36: Int = 10

In [38]:
sum(List("a", "b", "c"))

res37: String = "abc"

## Scalaz の型クラス

## メソッド注入 (enrich my library)

In [39]:
def plus[A: Monoid](a: A, b: A): A = implicitly[Monoid[A]].mappend(a, b)

defined function plus

In [40]:
plus(3, 4)

res39: Int = 7

In [41]:
trait MonoidOp[A] {
    val F: Monoid[A]
    val value: A
    def |+|(a2: A) = F.mappend(value, a2)
}

defined trait MonoidOp

In [42]:
implicit def toMonoidOp[A: Monoid](a: A): MonoidOp[A] = new MonoidOp[A] {
    val F = implicitly[Monoid[A]]
    val value = a
}

defined function toMonoidOp

In [43]:
3 |+| 4

res42: Int = 7

In [44]:
"a" |+| "b"

res43: String = "ab"

## 標準型構文

In [45]:
1.some | 2

res44: Int = 1

In [46]:
Some(1).getOrElse(2)

res45: Int = 1

In [47]:
(1 > 10) ? 1 | 2

res46: Int = 2

In [48]:
if (1 > 10) 1 else 2

res47: Int = 2